In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import scvelo as scv
import cellrank as cr
import anndata as ad
from cellrank.kernels import ConnectivityKernel, VelocityKernel
from cellrank.estimators import GPCCA
from tueplots import bundles
from tueplots import axes

In [3]:
plt.rcParams.update(**bundles.neurips2022(usetex=False, nrows=2), **axes.lines(base_width=0.5), **{"figure.dpi": 250})

scv.set_figure_params('scvelo', fontsize=20, color_map='viridis', dpi=400)  # for beautified visualization

cr.logging.print_versions()

cellrank==1.5.1 scanpy==1.9.1 anndata==0.8.0 numpy==1.23.5 numba==0.56.4 scipy==1.9.3 pandas==1.5.2 pygpcca==1.0.4 scikit-learn==1.1.3 statsmodels==0.13.5 scvelo==0.2.5 pygam==0.8.0 matplotlib==3.6.2 seaborn==0.12.1


In [4]:
adata = scv.datasets.gastrulation()

adata_25 = sc.pp.subsample(adata[adata.obs['stage'] == 'E8.25', :], n_obs=1000, copy=True)
adata_5 = sc.pp.subsample(adata[adata.obs['stage'] == 'E8.5', :], n_obs=1000, copy=True) 

del adata

In [5]:
scv.tl.recover_dynamics(adata_25, n_jobs=4)
scv.tl.velocity(adata_25, mode='dynamical')

recovering dynamics (using 4/8 cores)


  0%|          | 0/53801 [00:00<?, ?gene/s]

: 

: 

In [ ]:
scv.tl.recover_dynamics(adata_5, n_jobs=10)
scv.tl.velocity(adata_5, mode='dynamical')

Concatenate timepoints

In [ ]:
adata = ad.concat([adata_25, adata_5])
print(adata)

In [ ]:
scv.set_figure_params('scvelo', transparent=True, fontsize=10, color_map='viridis')
scv.pl.scatter(adata,  color_map='viridis')

Transition probabilities

In [ ]:
vk = VelocityKernel(adata).compute_transition_matrix()

Macrostate calculation

In [ ]:
g = GPCCA(vk)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

In [ ]:
g.compute_macrostates(n_states=8, cluster_key="celltype")
scv.set_figure_params('scvel o', transparent=True, fontsize=20, color_map='viridis')
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

Fate probabilities

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        "Acinar", "Ductal_1", "Ductal_2", "Alpha", "Beta", "Delta", "Epsilon", #celltypes pancreas, CHANGE!
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu', time_to_absorption='all')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

In [ ]:

scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=["Acinar", "Ductal_1", "Ductal_2", "Alpha", "Beta", "Delta", "Epsilon", ],
    cluster_key='celltype',
    clusters=['Acinar', 'Multipotent', 'Tip', 'Ductal', 'Ngn3 High early', 'Ngn3 High late', 'Fev+ Alpha', 'Alpha', 'Fev+ Beta', 'Beta', 'Fev+ Delta', 'Delta', 'Fev+ Epsilon', 'Epsilon'],
    figsize=(17, 6),
    title="",
    save="transprobs_pancreas1415_scvelo.png"
)

Velocity Stream Plot

In [ ]:
adata.uns["velocity_graph"] = vk.transition_matrix

scv.set_figure_params('scvelo', transparent=True, fontsize=10, dpi_save=400,color_map='viridis')
fig = plt.figure()
ax = scv.pl.velocity_embedding_stream(adata, basis="umap", smooth=0.5, title="", legend_loc="none", show=False)
legend =ax.legend(bbox_to_anchor=[1.25, 1.5], loc='upper center', ncol=6,frameon=True, prop={'size': 18})
#plt.savefig("velstream_pancreas1415_scvelo", dpi=800, bbox_inches='tight')

In [ ]:
scv.tl.velocity_confidence(adata)

In [ ]:
scv.set_figure_params('scvelo', fontsize=20)
scv.settings.presenter_view = False
scv.pl.scatter(adata, color='velocity_confidence',
               perc=[2, 98],
               cmap='gnuplot',
               vmid=0.75,
               rescale_color=[0.5, 1.0],
               size=15,
               title="",
               #save="velconf_pancreas1415_scvelo.png"
            )